In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap

import tensorflow as tf
from keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import EfficientNetB0, DenseNet121
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
import os

In [19]:
class DataLoader():
    """Data Loader class"""
    def __init__(self):
        super().__init__()
    
    def image_equalization(self, image):
        """Equalizes the histogram of an image"""
        # R, G, B = cv2.split(image)
        # output1_R = cv2.equalizeHist(R)
        # output1_G = cv2.equalizeHist(G)
        # output1_B = cv2.equalizeHist(B)
        # equ = cv2.merge((output1_R, output1_G, output1_B))
        return image

    def load_train_data(self, path):
        """Loads dataset from path"""
        self.train_datagen = ImageDataGenerator(
            rescale=1./255, 
            validation_split=0.1,
            preprocessing_function=None
            )
        return self.train_datagen.flow_from_directory(
        path,
        subset='training',
        target_size=(224, 224),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical')
        
    
    def load_val_data(self, path):
        """Loads dataset from path"""
        self.train_datagen = ImageDataGenerator(
            rescale=1./255, 
            validation_split=0.1,
            preprocessing_function=None)
        return self.train_datagen.flow_from_directory(
        path,
        subset='validation',
        target_size=(224, 224),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical')
    
    
    def load_test_data(self, path):
        """Loads dataset from path"""
        self.test_datagen = ImageDataGenerator(
            rescale=1./255,
            preprocessing_function=None)
        return self.test_datagen.flow_from_directory(
        path,
        target_size=(224, 224),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical')


In [6]:
def dense_net_model(input_shape, classes=7):
    """DenseNet model"""
    inputs = layers.Input(shape=input_shape)
    base_model = DenseNet121(
        input_shape=input_shape, include_top=True, weights=None, classes=classes
    )(inputs)
    # base_model.trainable = False
    model = models.Model(inputs, base_model)
    return model


def vgg16_model(input_shape, classes=7):
    """VGG16 model"""
    inputs = layers.Input(shape=input_shape)
    base_model = VGG16(
        input_shape=input_shape, include_top=True, weights=None, classes=classes
    )(inputs)
    # base_model.trainable = False
    model = models.Model(inputs, base_model)
    return model


def efficient_net_model(input_shape, classes=7):
    """EfficientNet model"""
    inputs = layers.Input(shape=input_shape)
    base_model = EfficientNetB0(
        input_shape=input_shape, include_top=True, weights=None, classes=classes
    )(inputs)
    # base_model.trainable = False
    model = models.Model(inputs, base_model)
    return model


def compile_model(model, optimizer, loss, metrics):
    """Compiles model"""
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics,
        weighted_metrics=None,
        run_eagerly=None,
        steps_per_execution=None,
        jit_compile=None,
    )
    return model

In [9]:
def main():
    data_loader = DataLoader()
    train_data = data_loader.load_train_data("../data/train/")
    val_data = data_loader.load_val_data("../data/train/")
    test_data = data_loader.load_test_data("../data/test/")
    
    model_name = 'dense_net'
    
    if model_name == "vgg16":
        model = vgg16_model(
            input_shape=(224, 224, 3),
            classes=7,
        )
    elif model_name == "efficient_net":
        model = efficient_net_model(
            input_shape=(224, 224, 3),
            classes=7,
        )
    else:
        model = dense_net_model(
            input_shape=(224, 224, 3),
            classes=7,
        )

    model = compile_model(
        model,
        optimizer='adamw',
        loss="categorical_crossentropy",
        metrics=["accuracy", "precision", "recall", "f1_score"]#cfg.train.metrics,
    )
    """Model callbacks"""
    earlystopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", mode="min", verbose=1, patience=5
    )
    checkpointer = tf.keras.callbacks.ModelCheckpoint(
        filepath=f"{model_name}.weights.h5",
        save_weights_only=True,
        save_best_only=True,
    )

    # Train the model

    history = model.fit(
        train_data,
        validation_data=val_data,
        validation_steps=5,
        epochs=25,
        batch_size=64,
        callbacks=[earlystopping, checkpointer],
    )

    # Evaluate the model
    model.save(f"{model_name}_model.hdf5")
    test_loss, test_accuracy = model.evaluate(test_data)
    print(f"Test accuracy: {test_accuracy}")

    # Log the confusion matrix
    y_pred = model.predict(test_data)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = test_data.classes
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=test_data.class_indices
    )
    disp.plot(cmap="viridis", values_format="d")
    plt.xticks(rotation=90)
    plt.title("Confusion Matrix")
    plt.savefig("confusion_matrix.png")

    # Log the classification report
    class_names = [k for k, v in test_data.class_indices.items()]
    report = classification_report(
        y_true, y_pred, target_names=class_names, output_dict=True
    )
    report = {k: v for k, v in report.items() if k in class_names}
    report = {k: {k2: round(v2, 2) for k2, v2 in v.items()} for k, v in report.items()}
    report = {
        k: dict(sorted(v.items(), key=lambda item: item[1], reverse=True))
        for k, v in report.items()
    }
    report = {
        k: {k2: v2 for k2, v2 in v.items() if k2 != "support"}
        for k, v in report.items()
    }
    report = {
        k: {k2: v2 for k2, v2 in v.items() if k2 != "macro avg"}
        for k, v in report.items()
    }
    report = {
        k: {k2: v2 for k2, v2 in v.items() if k2 != "weighted avg"}
        for k, v in report.items()
    }
    _model_history(history)


def _model_history(model_info):
    accuracy = model_info.history["accuracy"]
    val_accuracy = model_info.history["val_accuracy"]
    loss = model_info.history["loss"]
    val_loss = model_info.history["val_loss"]
    epochs = range(1, len(accuracy) + 1)
    plt.figure(figsize=(20, 10))
    plt.plot(epochs, accuracy, "g-", label="Training accuracy")
    plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
    plt.title("Training and validation accuracy")
    plt.grid()
    plt.savefig("accuracy.png", dpi=300, bbox_inches="tight")

    plt.legend()

    plt.figure(figsize=(20, 10))
    plt.plot(epochs, loss, "g-", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()
    plt.grid()
    plt.savefig("loss.png", bbox_inches="tight", dpi=300)

In [20]:
main()

Found 6048 images belonging to 7 classes.
Found 672 images belonging to 7 classes.
Found 280 images belonging to 7 classes.


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define Early Stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss', # Monitor the validation set loss
    patience=10,        # Number of epochs with no improvement after which training will be stopped
    verbose=1,          # To log the number of epoch after which training was stopped
    restore_best_weights=True # Restores model weights from the epoch with the best value of the monitored metric.
)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

# Rest of your code...

inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_COLOR))
outputs = EfficientNetB0(include_top=True, weights=None, classes=len(classes))(inputs)
efficient_net_model = tf.keras.Model(inputs, outputs)

efficient_net_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])




**Model Run**

In [ ]:
# Use validation_data parameter to provide validation data for each epoch
efficient_net_hist = efficient_net_model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val), callbacks=[early_stopping])

In [ ]:
efficient_net_model.summary()

**Training and Validation Loss**

In [ ]:
# Access the training and validation loss values from the history object
train_loss = efficient_net_hist.history['loss']
val_loss = efficient_net_hist.history['val_loss']

# Plot the training and validation loss curves with a logarithmic scale for the Y-axis
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (log scale)')
plt.title('Training and Validation Loss')
plt.legend()
save_fig("Eggplant_Training and Validation Loss_EfficientNET")
plt.show()


**Training and Validation Accuracy**

In [ ]:
# Access the training and validation accuracy values from the history object
train_acc = efficient_net_hist.history['accuracy']
val_acc = efficient_net_hist.history['val_accuracy']

# Plot the training and validation accuracy curves
epochs = range(1, len(train_acc) + 1)
plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
save_fig("Eggplant_Training and Validation Accuracy_EfficientNET")
plt.show()

**Classification Report**

In [ ]:
from sklearn.metrics import classification_report
efficient_acc = efficient_net_model.evaluate(X_test, y_test)[1]
print('EfficientNET model accuracy:', efficient_acc*100)
y_pred = efficient_net_model.predict(X_test)
y_classes = [np.argmax(element) for element in y_pred]

# Convert one-hot encoded true labels to labels
y_true_labels = np.argmax(y_test, axis=1)

# Generate classification report
class_report = classification_report(y_true_labels, y_classes, target_names=['Wilt Disease', 'Small Leaf Disease', 'Healthy Leaf', 'Insect Pest Disease', 'Mosaic Virus Disease', 'Leaf Spot Disease','White Mold Disease'])
print(class_report)

**Test Accuracy**

In [ ]:
efficient_acc = efficient_net_model.evaluate(X_test, y_test)[1]
print('EfficientNET TEST accuracy:', efficient_acc*100)

**Confusion Matrix**

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Evaluate the model on the test set and get predictions
y_pred = efficient_net_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)  # Convert one-hot encoded predictions to labels

# Convert one-hot encoded true labels to labels
y_true_labels = np.argmax(y_test, axis=1)

# Generate confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Wilt Disease', 'Small Leaf Disease', 'Healthy Leaf', 'Insect Pest Disease', 'Mosaic Virus Disease', 'Leaf Spot Disease','White Mold Disease'])
disp.plot(cmap='viridis', values_format='d')
plt.xticks(rotation=90)
plt.title('Confusion Matrix')
save_fig("CM_EfficientNET")
plt.show()


**Predicting Images**

In [ ]:
save_fig("Result_EfficientNET")
show_images(images=X_test, actual_classes=y_test, predicted_classes=y_classes, test_nums=[1, 40, 100, 70]) # Test Cases: 1, 40, 100, 70

# VGG

**Early Stopping**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define Early Stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss', # Monitor the validation set loss
    patience=10,        # Number of epochs with no improvement after which training will be stopped
    verbose=1,          # To log the number of epoch after which training was stopped
    restore_best_weights=True # Restores model weights from the epoch with the best value of the monitored metric.
)


**Model Code**

In [ ]:
inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_COLOR))
outputs = VGG16(include_top=True, weights=None, classes=len(classes))(inputs)
vgg_model = tf.keras.Model(inputs, outputs)

vgg_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

**Model Run**

In [ ]:
vgg_hist = vgg_model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val), callbacks=[early_stopping])

**Model Summary**

In [ ]:
vgg_model.summary()

**Training and Validation Loss**

In [ ]:
# Access the training and validation loss values from the history object
train_loss = vgg_hist.history['loss']
val_loss = vgg_hist.history['val_loss']

# Plot the training and validation loss curves with a logarithmic scale for the Y-axis
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (log scale)')
plt.title('Training and Validation Loss')
plt.legend()
save_fig("Eggplant_Training and Validation Loss_VGG")
plt.show()


**Training and Validation Accuracy**

In [ ]:
# Access the training and validation accuracy values from the history object
train_acc = vgg_hist.history['accuracy']
val_acc = vgg_hist.history['val_accuracy']

# Plot the training and validation accuracy curves
epochs = range(1, len(train_acc) + 1)
plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
save_fig("Eggplant_Training and Validation Accuracy_VGG")
plt.show()

**Classification Report**

In [ ]:
from sklearn.metrics import classification_report
vgg_acc = vgg_model.evaluate(X_test, y_test)[1]
print('VGG model accuracy:', vgg_acc*100)
y_pred = efficient_net_model.predict(X_test)
y_classes = [np.argmax(element) for element in y_pred]

# Convert one-hot encoded true labels to labels
y_true_labels = np.argmax(y_test, axis=1)

# Generate classification report
class_report = classification_report(y_true_labels, y_classes, target_names=['Wilt Disease', 'Small Leaf Disease', 'Healthy Leaf', 'Insect Pest Disease', 'Mosaic Virus Disease', 'Leaf Spot Disease','White Mold Disease'])
print(class_report)

**Test Accuracy**

In [ ]:
vgg_acc = vgg_model.evaluate(X_test, y_test)[1]
print('VGG TEST accuracy:', vgg_acc*100)

**Confusion Matrix**

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Evaluate the model on the test set and get predictions
y_pred = vgg_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)  # Convert one-hot encoded predictions to labels

# Convert one-hot encoded true labels to labels
y_true_labels = np.argmax(y_test, axis=1)

# Generate confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Wilt Disease', 'Small Leaf Disease', 'Healthy Leaf', 'Insect Pest Disease', 'Mosaic Virus Disease', 'Leaf Spot Disease','White Mold Disease'])
disp.plot(cmap='viridis', values_format='d')
plt.xticks(rotation=90)
plt.title('Confusion Matrix')
save_fig("CM_VGG")
plt.show()


**Predicting Images**

In [ ]:
save_fig("Result_VGGNET")
show_images(images=X_test, actual_classes=y_test, predicted_classes=y_classes, test_nums=[1, 40, 100, 70]) # Test Cases: 1, 40, 100, 70

# Vision Transformer

In [ ]:
import tensorflow_addons as tfa

In [ ]:
X, y = data_preparation(DATASET_DIR, classes)
EQU_IMAGES = image_equalization(ALL_IMAGES)
print('Number of images: ', len(ALL_IMAGES))

In [ ]:
EQU_IMAGES = np.array(EQU_IMAGES)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(EQU_IMAGES, y, test_size=0.22, random_state=42)
print(f"x_train shape: {X_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {X_test.shape} - y_test shape: {y_test.shape}")

In [ ]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_COLOR)
num_classes = 7
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 120
image_size = 72 # image resized to 72, 72
patch_size = 6
num_patches = (image_size // patch_size) ** 2

projection_dim = 4 # dimension vectors
num_heads = 4
transformer_units = [projection_dim * 2, projection_dim,]
transformer_layers = 8
mlp_head_units = [2048, 1024]

In [ ]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.Normalization(),
    layers.experimental.preprocessing.Resizing(image_size, image_size),
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(factor=0.02),
    layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),

],
    name='layers'
)

data_augmentation.layers[0].adapt(X_train)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
plt.figure(figsize=(4, 4))
image = X_train[np.random.choice(range(X_train.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, accuracy

In [ ]:
from sklearn.metrics import classification_report

def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    report = classification_report(y_test, y_pred)

    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    print("Classification Report:")
    print(report)

    return history, accuracy, report


In [ ]:
vit_classifier = create_vit_classifier()

In [ ]:
history, vit_accuracy = run_experiment(vit_classifier)

# Swin Transformer

In [ ]:
ALL_IMAGES, LABELS = data_preparation(DATASET_DIR, classes)
EQU_IMAGES = image_equalization(ALL_IMAGES)
print('Number of images: ', len(ALL_IMAGES))

In [ ]:
onehot_encoded = hot_encoder(LABELS)
X, y = EQU_IMAGES, onehot_encoded

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.22, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_COLOR)
num_classes = 7
patch_size = (2, 2)  # 2-by-2 sized patches
dropout_rate = 0.03  # Dropout rate
num_heads = 8  # Attention heads
embed_dim = 64  # Embedding dimension
num_mlp = 256  # MLP layer size
qkv_bias = True  # Convert embedded patches to query, key, and values with a learnable additive value
window_size = 2  # Size of attention window
shift_size = 1  # Size of shifting window
image_dimension = input_shape[0]  # Initial image size

num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]

learning_rate = 1e-3
batch_size = 32
num_epochs = 40
validation_split = 0.1
weight_decay = 0.0001
label_smoothing = 0.1

In [ ]:
def window_partition(x, window_size):
    _, height, width, channels = x.shape
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        x, shape=(-1, patch_num_y, window_size, patch_num_x, window_size, channels)
    )
    x = tf.transpose(x, (0, 1, 3, 2, 4, 5))
    windows = tf.reshape(x, shape=(-1, window_size, window_size, channels))
    return windows


def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        windows,
        shape=(-1, patch_num_y, patch_num_x, window_size, window_size, channels),
    )
    x = tf.transpose(x, perm=(0, 1, 3, 2, 4, 5))
    x = tf.reshape(x, shape=(-1, height, width, channels))
    return x


class DropPath(layers.Layer):
    def __init__(self, drop_prob=None, **kwargs):
        super(DropPath, self).__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, x):
        input_shape = tf.shape(x)
        batch_size = input_shape[0]
        rank = x.shape.rank
        shape = (batch_size,) + (1,) * (rank - 1)
        random_tensor = (1 - self.drop_prob) + tf.random.uniform(shape, dtype=x.dtype)
        path_mask = tf.floor(random_tensor)
        output = tf.math.divide(x, 1 - self.drop_prob) * path_mask
        return output

In [ ]:
class WindowAttention(layers.Layer):
    def __init__(
        self, dim, window_size, num_heads, qkv_bias=True, dropout_rate=0.0, **kwargs
    ):
        super(WindowAttention, self).__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = layers.Dense(dim * 3, use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

    def build(self, input_shape):
        num_window_elements = (2 * self.window_size[0] - 1) * (
            2 * self.window_size[1] - 1
        )
        self.relative_position_bias_table = self.add_weight(
            shape=(num_window_elements, self.num_heads),
            initializer=tf.initializers.Zeros(),
            trainable=True,
        )
        coords_h = np.arange(self.window_size[0])
        coords_w = np.arange(self.window_size[1])
        coords_matrix = np.meshgrid(coords_h, coords_w, indexing="ij")
        coords = np.stack(coords_matrix)
        coords_flatten = coords.reshape(2, -1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)

        self.relative_position_index = tf.Variable(
            initial_value=tf.convert_to_tensor(relative_position_index), trainable=False
        )

    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, 3, self.num_heads, head_dim))
        x_qkv = tf.transpose(x_qkv, perm=(2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = tf.transpose(k, perm=(0, 1, 3, 2))
        attn = q @ k

        num_window_elements = self.window_size[0] * self.window_size[1]
        relative_position_index_flat = tf.reshape(
            self.relative_position_index, shape=(-1,)
        )
        relative_position_bias = tf.gather(
            self.relative_position_bias_table, relative_position_index_flat
        )
        relative_position_bias = tf.reshape(
            relative_position_bias, shape=(num_window_elements, num_window_elements, -1)
        )
        relative_position_bias = tf.transpose(relative_position_bias, perm=(2, 0, 1))
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.get_shape()[0]
            mask_float = tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), tf.float32
            )
            attn = (
                tf.reshape(attn, shape=(-1, nW, self.num_heads, size, size))
                + mask_float
            )
            attn = tf.reshape(attn, shape=(-1, self.num_heads, size, size))
            attn = keras.activations.softmax(attn, axis=-1)
        else:
            attn = keras.activations.softmax(attn, axis=-1)
        attn = self.dropout(attn)

        x_qkv = attn @ v
        x_qkv = tf.transpose(x_qkv, perm=(0, 2, 1, 3))
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv

In [ ]:
class SwinTransformer(layers.Layer):
    def __init__(
        self,
        dim,
        num_patch,
        num_heads,
        window_size=7,
        shift_size=0,
        num_mlp=1024,
        qkv_bias=True,
        dropout_rate=0.0,
        **kwargs,
    ):
        super(SwinTransformer, self).__init__(**kwargs)

        self.dim = dim  # number of input dimensions
        self.num_patch = num_patch  # number of embedded patches
        self.num_heads = num_heads  # number of attention heads
        self.window_size = window_size  # size of window
        self.shift_size = shift_size  # size of window shift
        self.num_mlp = num_mlp  # number of MLP nodes

        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.attn = WindowAttention(
            dim,
            window_size=(self.window_size, self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            dropout_rate=dropout_rate,
        )
        self.drop_path = DropPath(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

        self.mlp = keras.Sequential(
            [
                layers.Dense(num_mlp),
                layers.Activation(keras.activations.gelu),
                layers.Dropout(dropout_rate),
                layers.Dense(dim),
                layers.Dropout(dropout_rate),
            ]
        )

        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size == 0:
            self.attn_mask = None
        else:
            height, width = self.num_patch
            h_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            w_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            mask_array = np.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_array = tf.convert_to_tensor(mask_array)

            # mask array to windows
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = tf.reshape(
                mask_windows, shape=[-1, self.window_size * self.window_size]
            )
            attn_mask = tf.expand_dims(mask_windows, axis=1) - tf.expand_dims(
                mask_windows, axis=2
            )
            attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
            attn_mask = tf.where(attn_mask == 0, 0.0, attn_mask)
            self.attn_mask = tf.Variable(initial_value=attn_mask, trainable=False)

    def call(self, x):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = tf.reshape(x, shape=(-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = tf.roll(
                x, shift=[-self.shift_size, -self.shift_size], axis=[1, 2]
            )
        else:
            shifted_x = x

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = tf.reshape(
            x_windows, shape=(-1, self.window_size * self.window_size, channels)
        )
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        attn_windows = tf.reshape(
            attn_windows, shape=(-1, self.window_size, self.window_size, channels)
        )
        shifted_x = window_reverse(
            attn_windows, self.window_size, height, width, channels
        )
        if self.shift_size > 0:
            x = tf.roll(
                shifted_x, shift=[self.shift_size, self.shift_size], axis=[1, 2]
            )
        else:
            x = shifted_x

        x = tf.reshape(x, shape=(-1, height * width, channels))
        x = self.drop_path(x)
        x = x_skip + x
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = x_skip + x
        return x

In [ ]:
class PatchExtract(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(PatchExtract, self).__init__(**kwargs)
        self.patch_size_x = patch_size[0]
        self.patch_size_y = patch_size[0]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=(1, self.patch_size_x, self.patch_size_y, 1),
            strides=(1, self.patch_size_x, self.patch_size_y, 1),
            rates=(1, 1, 1, 1),
            padding="VALID",
        )
        patch_dim = patches.shape[-1]
        patch_num = patches.shape[1]
        return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))

In [ ]:
class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super(PatchEmbedding, self).__init__(**kwargs)
        self.num_patch = num_patch
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patch, output_dim=embed_dim)

    def call(self, patch):
        pos = tf.range(start=0, limit=self.num_patch, delta=1)
        return self.proj(patch) + self.pos_embed(pos)

In [ ]:
class PatchMerging(tf.keras.layers.Layer):
    def __init__(self, num_patch, embed_dim):
        super(PatchMerging, self).__init__()
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.get_shape().as_list()
        x = tf.reshape(x, shape=(-1, height, width, C))
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = tf.concat((x0, x1, x2, x3), axis=-1)
        x = tf.reshape(x, shape=(-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)

In [ ]:
input = layers.Input(input_shape)
x = layers.RandomCrop(image_dimension, image_dimension)(input)
x = layers.RandomFlip("horizontal")(x)
x = PatchExtract(patch_size)(x)
x = PatchEmbedding(num_patch_x * num_patch_y, embed_dim)(x)
x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=0,
    num_mlp=num_mlp,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
)(x)
x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=shift_size,
    num_mlp=num_mlp,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
)(x)
x = PatchMerging((num_patch_x, num_patch_y), embed_dim=embed_dim)(x)
x = layers.GlobalAveragePooling1D()(x)
output = layers.Dense(num_classes, activation="softmax")(x)

In [ ]:
import tensorflow_addons as tfa

In [ ]:
model = keras.Model(input, output)
model.compile(
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
    optimizer=tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=validation_split,
)

In [ ]:
loss, swin_accuracy= model.evaluate(X_test, y_test)

In [ ]:
swin_accuracy